In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import drive.MyDrive.Proyecto.project_utils as pu
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

In [4]:
BATCH_SIZE = 64

# Modelos

In [5]:
import drive.MyDrive.Proyecto.models as mdls

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHANNELS = 39
BASEPATH = "/content/drive/MyDrive/Proyecto/Datasets/"
pu.iniciar_semillas()

In [9]:
windows = [20, 40, 80, 120]
bidirecc = [True, False]
data= [
    #(learning_rate, epochs, hidden_size, num_layers, modelo)
    (2e-4, 100, 256, 1, mdls.RNN),
    (2e-4, 100, 64, 2, mdls.GRU),
    (2e-4, 100, 64, 2, mdls.LSTM),
]
BATCH_SIZE = 64

for window in windows:
  train_df = pd.read_pickle(BASEPATH + f'train_{window}ms_win.pkl')
  val_df = pd.read_pickle(BASEPATH + f'val_{window}ms_win.pkl')
  train_df = pu.oversample(train_df)
  max_length = max(
      train_df.mfcc.apply(lambda x: x[0].size).max(),
      train_df.delta.apply(lambda x: x[0].size).max(),
      val_df.mfcc.apply(lambda x: x[0].size).max(),
      val_df.delta.apply(lambda x: x[0].size).max()
  )
  categories, c2i, i2c = pu.get_categories(train_df)
  NUM_CLASSES = len(categories)
  train_df, test_df = train_test_split(train_df, test_size=0.15)
  
  train_data = pu.SCData(train_df, categories, c2i, i2c, max_length)
  test_data = pu.SCData(test_df, categories, c2i, i2c, max_length)
  valid_data = pu.SCData(val_df, categories, c2i, i2c, max_length)

  train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
  test_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
  valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
  for bidirec in bidirecc:
    for datum in data:
      lr, epochs, hs, n_lay, cls = datum
      name = cls.__name__
      desc = f'_{window}ms_{bidirec}'
      model_name = f'{name}_{desc}'
      model = cls(hs, n_lay, NUM_CLASSES, CHANNELS, max_length, bidirec)
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)  

      train_loss_evolution, train_acc_evolution, val_acc_evolution, val_loss_evolution = pu.train_complete(model, criterion, optimizer, epochs, train_loader, valid_loader, model_name, DEVICE, False)

      model = cls(hs, n_lay, NUM_CLASSES, CHANNELS, max_length, bidirec)
      model.load_state_dict(torch.load(f'{model_name}.pt'))

      # pu.plot_train_evo(train_loss_evolution, val_loss_evolution, train_acc_evolution, val_acc_evolution, name, desc)
      # pu.plot_conf_matrix(model, valid_loader, categories, name, desc, DEVICE)



100%|██████████| 9981/9981 [00:02<00:00, 4142.59it/s]
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 3.549 | Train Acc: 3.83%
	 Val. Loss: 3.552 |  Val. Acc: 3.42%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 3.533 | Train Acc: 4.53%
	 Val. Loss: 3.555 |  Val. Acc: 2.51%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 3.516 | Train Acc: 5.01%
	 Val. Loss: 3.538 |  Val. Acc: 4.30%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 3.519 | Train Acc: 5.61%
	 Val. Loss: 3.552 |  Val. Acc: 3.68%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 3.481 | Train Acc: 6.44%
	 Val. Loss: 3.525 |  Val. Acc: 3.84%
Epoch: 06 | Epoch Time: 0m 2s
	Train Loss: 3.414 | Train Acc: 7.76%
	 Val. Loss: 3.459 |  Val. Acc: 4.78%
Epoch: 07 | Epoch Time: 0m 2s
	Train Loss: 3.301 | Train Acc: 9.71%
	 Val. Loss: 3.351 |  Val. Acc: 6.64%
Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 3.317 | Train Acc: 8.24%
	 Val. Loss: 3.355 |  Val. Acc: 6.59%
Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 3.296 | Train Acc: 9.14%
	 Val. Loss: 3.351 |  Val. Acc: 7.17%
Epoch: 10 | Epoch Time: 0m 2s
	Train Loss: 3.2

100%|██████████| 9981/9981 [00:01<00:00, 5179.92it/s]
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 3.335 | Train Acc: 8.16%
	 Val. Loss: 3.364 |  Val. Acc: 6.10%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 3.211 | Train Acc: 10.64%
	 Val. Loss: 3.238 |  Val. Acc: 8.99%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 3.072 | Train Acc: 13.42%
	 Val. Loss: 3.106 |  Val. Acc: 11.12%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 2.988 | Train Acc: 14.51%
	 Val. Loss: 3.021 |  Val. Acc: 12.19%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 2.841 | Train Acc: 18.60%
	 Val. Loss: 2.908 |  Val. Acc: 14.29%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 2.698 | Train Acc: 20.54%
	 Val. Loss: 2.778 |  Val. Acc: 16.36%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 2.699 | Train Acc: 20.82%
	 Val. Loss: 2.763 |  Val. Acc: 16.81%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 2.583 | Train Acc: 23.72%
	 Val. Loss: 2.628 |  Val. Acc: 19.83%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 2.435 | Train Acc: 27.52%
	 Val. Loss: 2.517 |  Val. Acc: 22.74%
Epoch: 10 | Epoch Time: 0m 1s
	

100%|██████████| 9981/9981 [00:02<00:00, 4161.90it/s]
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 3.039 | Train Acc: 13.74%
	 Val. Loss: 3.059 |  Val. Acc: 11.64%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 2.809 | Train Acc: 18.69%
	 Val. Loss: 2.886 |  Val. Acc: 14.92%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 2.702 | Train Acc: 21.38%
	 Val. Loss: 2.783 |  Val. Acc: 17.94%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 2.592 | Train Acc: 22.58%
	 Val. Loss: 2.637 |  Val. Acc: 20.57%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 2.447 | Train Acc: 27.03%
	 Val. Loss: 2.542 |  Val. Acc: 22.69%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 2.326 | Train Acc: 30.12%
	 Val. Loss: 2.419 |  Val. Acc: 24.69%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 2.394 | Train Acc: 26.47%
	 Val. Loss: 2.407 |  Val. Acc: 23.94%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 2.125 | Train Acc: 36.08%
	 Val. Loss: 2.232 |  Val. Acc: 30.61%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 2.132 | Train Acc: 35.36%
	 Val. Loss: 2.217 |  Val. Acc: 30.81%
Epoch: 10 | Epoch Time: 0m 1

100%|██████████| 9981/9981 [00:01<00:00, 5417.33it/s]
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 2.879 | Train Acc: 19.30%
	 Val. Loss: 2.924 |  Val. Acc: 17.74%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 2.645 | Train Acc: 24.43%
	 Val. Loss: 2.715 |  Val. Acc: 20.71%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 2.568 | Train Acc: 25.90%
	 Val. Loss: 2.701 |  Val. Acc: 21.04%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 2.289 | Train Acc: 33.12%
	 Val. Loss: 2.373 |  Val. Acc: 29.39%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 2.179 | Train Acc: 37.01%
	 Val. Loss: 2.256 |  Val. Acc: 33.77%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 1.948 | Train Acc: 44.13%
	 Val. Loss: 2.075 |  Val. Acc: 38.64%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 1.933 | Train Acc: 41.96%
	 Val. Loss: 2.029 |  Val. Acc: 38.71%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 1.711 | Train Acc: 50.01%
	 Val. Loss: 1.858 |  Val. Acc: 44.55%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 1.613 | Train Acc: 53.61%
	 Val. Loss: 1.772 |  Val. Acc: 48.20%
Epoch: 10 | Epoch Time: 0m 1